In [1]:
import numpy as np
import pandas as pd

In [2]:
d = pd.read_csv('MTA_DATA.csv')

## Exploratory Data Analysis

In [3]:
print(d.shape)
print(d.dtypes)
print(d.describe())
d.head()

(10133871, 7)
RecordID                int64
Visitor_key             int64
Hit_date               object
Order_ind               int64
CONTAINER_ID          float64
Channel                object
Min_Visit Rtrn Cnt      int64
dtype: object
           RecordID   Visitor_key     Order_ind  CONTAINER_ID  \
count  1.013387e+07  1.013387e+07  1.013387e+07  3.986900e+05   
mean   5.066936e+06  1.889485e+08  2.892695e-02  1.219439e+08   
std    2.925397e+06  3.977088e+07  1.676013e-01  1.700850e+07   
min    1.000000e+00  3.038000e+03  0.000000e+00  1.456480e+06   
25%    2.533468e+06  1.875366e+08  0.000000e+00  1.230413e+08   
50%    5.066936e+06  2.057652e+08  0.000000e+00  1.266599e+08   
75%    7.600404e+06  2.111246e+08  0.000000e+00  1.307305e+08   
max    1.013387e+07  2.134299e+08  1.000000e+00  1.349686e+08   

       Min_Visit Rtrn Cnt  
count        1.013387e+07  
mean         2.959447e+01  
std          7.948659e+01  
min          1.000000e+00  
25%          1.000000e+00  
50%     

,RecordID,Visitor_key,Hit_date,Order_ind,CONTAINER_ID,Channel,Min_Visit Rtrn Cnt
0,1,100004047,2018-11-06,0,NaN,Paid Search,39
1,2,100004047,2018-11-07,0,NaN,Paid Search,39
2,3,100004047,2018-11-13,0,NaN,Paid Search,39
3,4,100004047,2018-11-14,0,NaN,Paid Search,39
4,5,100004047,2018-11-07,0,NaN,SEO,41


In [4]:
# Data type clean up
d['Visitor_key'] = d['Visitor_key'].astype('str')
d['CONTAINER_ID'] = d['CONTAINER_ID'].astype('str')
d['Order_ind'] = d['Order_ind'].astype('str')
d['Hit_date'] = d['Hit_date'].astype('datetime64[ns]')
print(d.dtypes)

RecordID                       int64
Visitor_key                   object
Hit_date              datetime64[ns]
Order_ind                     object
CONTAINER_ID                  object
Channel                       object
Min_Visit Rtrn Cnt             int64
dtype: object


In [5]:
# Check for nulls
d.isna().sum()

RecordID              0
Visitor_key           0
Hit_date              0
Order_ind             0
CONTAINER_ID          0
Channel               0
Min_Visit Rtrn Cnt    0
dtype: int64

In [6]:
d.groupby('Channel').count()

,RecordID,Visitor_key,Hit_date,Order_ind,CONTAINER_ID,Min_Visit Rtrn Cnt
Channel,,,,,,
Digital Event,131,131,131,131,131,131
Direct+Other,204203,204203,204203,204203,204203,204203
Display,884815,884815,884815,884815,884815,884815
Email,279870,279870,279870,279870,279870,279870
Friendly URL,272629,272629,272629,272629,272629,272629
Paid Search,2002360,2002360,2002360,2002360,2002360,2002360
SEO,4371116,4371116,4371116,4371116,4371116,4371116
Social,255328,255328,255328,255328,255328,255328
Social-Paid,29844,29844,29844,29844,29844,29844


In [7]:
# Clean up the channel "Unknown" it is mislebeled
d['Channel'] = d['Channel'].str.replace('Unknown','Direct+Other')
d.groupby('Channel').count()

,RecordID,Visitor_key,Hit_date,Order_ind,CONTAINER_ID,Min_Visit Rtrn Cnt
Channel,,,,,,
Digital Event,131,131,131,131,131,131
Direct+Other,2037778,2037778,2037778,2037778,2037778,2037778
Display,884815,884815,884815,884815,884815,884815
Email,279870,279870,279870,279870,279870,279870
Friendly URL,272629,272629,272629,272629,272629,272629
Paid Search,2002360,2002360,2002360,2002360,2002360,2002360
SEO,4371116,4371116,4371116,4371116,4371116,4371116
Social,255328,255328,255328,255328,255328,255328
Social-Paid,29844,29844,29844,29844,29844,29844


In [8]:
# Limiting 10M records to 10,000 to make it easier to work
# I will open this up later...

#d = d.head(10000)

In [9]:
#Create consistent level of detail within the data

d['grp_fld'] = d.Visitor_key + d.CONTAINER_ID+ d.Order_ind

##### Establish Touch Sequences within each journey

In [10]:
dfa = d.groupby(['grp_fld','Channel','Visitor_key','CONTAINER_ID','Order_ind']).agg({'Min_Visit Rtrn Cnt': 'min'})

dfa = dfa.sort_values(['grp_fld','Min_Visit Rtrn Cnt']).reset_index()

dfa["TouchSeq"] = dfa.groupby('grp_fld')['Min_Visit Rtrn Cnt'].rank(method='dense', ascending=True).astype(int)
dfa.head(10)

,grp_fld,Channel,Visitor_key,CONTAINER_ID,Order_ind,Min_Visit Rtrn Cnt,TouchSeq
0,100004047nan0,Paid Search,100004047,nan,0,39,1
1,100004047nan0,SEO,100004047,nan,0,41,2
2,100004089nan0,Paid Search,100004089,nan,0,65,1
3,100004089nan0,SEO,100004089,nan,0,67,2
4,100004330nan0,SEO,100004330,nan,0,1,1
5,100004606114473205.01,Direct+Other,100004606,114473205.0,1,49,1
6,100004606114473205.01,SEO,100004606,114473205.0,1,50,2
7,100004606123853417.01,Direct+Other,100004606,123853417.0,1,49,1
8,100004606123853417.01,SEO,100004606,123853417.0,1,50,2
9,100004606128357685.01,Direct+Other,100004606,128357685.0,1,49,1


##### Establish a starting point for each journey (assuming the SP is offsite)

In [11]:
dfb = d[['grp_fld','Visitor_key','CONTAINER_ID','Order_ind']]
dfb = dfb.drop_duplicates()
dfb['Min_Visit Rtrn Cnt'] = -1 #set to the beginning of the sort order
dfb['TouchSeq'] = 0
dfb['Channel'] = 'Start'

print(dfb.shape)
print(dfb.dtypes)
print(dfb.describe())
dfb.head()

(3972992, 7)
grp_fld               object
Visitor_key           object
CONTAINER_ID          object
Order_ind             object
Min_Visit Rtrn Cnt     int64
TouchSeq               int64
Channel               object
dtype: object
       Min_Visit Rtrn Cnt   TouchSeq
count           3972992.0  3972992.0
mean                 -1.0        0.0
std                   0.0        0.0
min                  -1.0        0.0
25%                  -1.0        0.0
50%                  -1.0        0.0
75%                  -1.0        0.0
max                  -1.0        0.0


,grp_fld,Visitor_key,CONTAINER_ID,Order_ind,Min_Visit Rtrn Cnt,TouchSeq,Channel
0,100004047nan0,100004047,nan,0,-1,0,Start
7,100004089nan0,100004089,nan,0,-1,0,Start
26,100004330nan0,100004330,nan,0,-1,0,Start
27,100004713nan0,100004713,nan,0,-1,0,Start
36,100005387nan0,100005387,nan,0,-1,0,Start


##### Establish ending point for each journey

In [12]:
dfc = dfa.groupby(['grp_fld','Visitor_key','CONTAINER_ID','Order_ind']).agg({'TouchSeq': 'max'})
dfc = dfc.reset_index()

# use function to determine if order was created
def chan(row):
    if row['Order_ind'] == "1":
        return 'Conv'
    else:
        return 'End'
dfc['Channel'] = dfc.apply(chan, axis=1)

#set final touch sequence number
dfc['TouchSeq'] = dfc['TouchSeq'] + 1

dfc.head()

,grp_fld,Visitor_key,CONTAINER_ID,Order_ind,TouchSeq,Channel
0,100004047nan0,100004047,nan,0,3,End
1,100004089nan0,100004089,nan,0,3,End
2,100004330nan0,100004330,nan,0,2,End
3,100004606114473205.01,100004606,114473205.0,1,3,Conv
4,100004606123853417.01,100004606,123853417.0,1,3,Conv


In [13]:
frames = [dfa, dfb, dfc]

dff = pd.concat(frames, axis=0, join='outer', join_axes=None, ignore_index=True,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True, sort=True)

dff.shape
dff.sort_values(by=['grp_fld','TouchSeq'] ,inplace=True)
dff.head(100)

,CONTAINER_ID,Channel,Min_Visit Rtrn Cnt,Order_ind,TouchSeq,Visitor_key,grp_fld
4862989,nan,Start,-1.0,0,0,100004047,100004047nan0
0,nan,Paid Search,39.0,0,1,100004047,100004047nan0
1,nan,SEO,41.0,0,2,100004047,100004047nan0
8835981,nan,End,NaN,0,3,100004047,100004047nan0
4862990,nan,Start,-1.0,0,0,100004089,100004089nan0
2,nan,Paid Search,65.0,0,1,100004089,100004089nan0
3,nan,SEO,67.0,0,2,100004089,100004089nan0
8835982,nan,End,NaN,0,3,100004089,100004089nan0
4862991,nan,Start,-1.0,0,0,100004330,100004330nan0
4,nan,SEO,1.0,0,1,100004330,100004330nan0


In [14]:
dff.shape

(12808973, 7)

In [15]:
# Select relevant columns for Markov Chain Analysis
journeys = dff[['grp_fld','Visitor_key','Order_ind','CONTAINER_ID','TouchSeq','Channel']]
journeys.reset_index(drop=True, inplace=True)

# not used, just a thought..
#journeys['Path'] = journeys.groupby(['grp_fld','Visitor_key','Order_ind','CONTAINER_ID'])['Channel'].transform(lambda x: '>'.join(x))


In [16]:
conditions = ['End','Conv']
journeys['Transition'] = np.where(journeys['Channel'].isin(conditions),np.nan ,journeys['Channel'].shift(-1))
#journeys['Channel'] = journeys['Channel'].shift(-1)
journeys.head(50)

,grp_fld,Visitor_key,Order_ind,CONTAINER_ID,TouchSeq,Channel,Transition
0,100004047nan0,100004047,0,nan,0,Start,Paid Search
1,100004047nan0,100004047,0,nan,1,Paid Search,SEO
2,100004047nan0,100004047,0,nan,2,SEO,End
3,100004047nan0,100004047,0,nan,3,End,NaN
4,100004089nan0,100004089,0,nan,0,Start,Paid Search
5,100004089nan0,100004089,0,nan,1,Paid Search,SEO
6,100004089nan0,100004089,0,nan,2,SEO,End
7,100004089nan0,100004089,0,nan,3,End,NaN
8,100004330nan0,100004330,0,nan,0,Start,SEO
9,100004330nan0,100004330,0,nan,1,SEO,End


In [17]:
# Filter the End and Conv from channel
# this represents the starting point from which a customer transtions to another channel
# these two conditions have no transition

journeys = journeys.loc[(journeys.Channel != 'End') & (journeys.Channel != 'Conv')]
journeys.head(20)

,grp_fld,Visitor_key,Order_ind,CONTAINER_ID,TouchSeq,Channel,Transition
0,100004047nan0,100004047,0,nan,0,Start,Paid Search
1,100004047nan0,100004047,0,nan,1,Paid Search,SEO
2,100004047nan0,100004047,0,nan,2,SEO,End
4,100004089nan0,100004089,0,nan,0,Start,Paid Search
5,100004089nan0,100004089,0,nan,1,Paid Search,SEO
6,100004089nan0,100004089,0,nan,2,SEO,End
8,100004330nan0,100004330,0,nan,0,Start,SEO
9,100004330nan0,100004330,0,nan,1,SEO,End
11,100004606114473205.01,100004606,1,114473205.0,0,Start,Direct+Other
12,100004606114473205.01,100004606,1,114473205.0,1,Direct+Other,SEO


In [18]:
# Create lookup table with total number of transitions per Channel
initial_channel_counts = journeys.groupby('Channel').agg ({'Transition': 'count'}).reset_index()
initial_channel_counts.rename(columns={'Transition':'Total_transitions'}, inplace=True)
initial_channel_counts.columns

Index([u'Channel', u'Total_transitions'], dtype='object')

In [19]:
# Join the lookup table to the transitions table, to pull in the counts for each starting state, t.
Transitions = pd.merge(journeys, initial_channel_counts, on='Channel', how='left')
Transitions.head(100)

,grp_fld,Visitor_key,Order_ind,CONTAINER_ID,TouchSeq,Channel,Transition,Total_transitions
0,100004047nan0,100004047,0,nan,0,Start,Paid Search,3972992
1,100004047nan0,100004047,0,nan,1,Paid Search,SEO,894252
2,100004047nan0,100004047,0,nan,2,SEO,End,1976015
3,100004089nan0,100004089,0,nan,0,Start,Paid Search,3972992
4,100004089nan0,100004089,0,nan,1,Paid Search,SEO,894252
5,100004089nan0,100004089,0,nan,2,SEO,End,1976015
6,100004330nan0,100004330,0,nan,0,Start,SEO,3972992
7,100004330nan0,100004330,0,nan,1,SEO,End,1976015
8,100004606114473205.01,100004606,1,114473205.0,0,Start,Direct+Other,3972992
9,100004606114473205.01,100004606,1,114473205.0,1,Direct+Other,SEO,1281018


In [20]:
Transitions['probability'] = 1/Transitions['Total_transitions']
Transitions.head(100)

,grp_fld,Visitor_key,Order_ind,CONTAINER_ID,TouchSeq,Channel,Transition,Total_transitions,probability
0,100004047nan0,100004047,0,nan,0,Start,Paid Search,3972992,2.516995e-07
1,100004047nan0,100004047,0,nan,1,Paid Search,SEO,894252,1.118253e-06
2,100004047nan0,100004047,0,nan,2,SEO,End,1976015,5.060690e-07
3,100004089nan0,100004089,0,nan,0,Start,Paid Search,3972992,2.516995e-07
4,100004089nan0,100004089,0,nan,1,Paid Search,SEO,894252,1.118253e-06
5,100004089nan0,100004089,0,nan,2,SEO,End,1976015,5.060690e-07
6,100004330nan0,100004330,0,nan,0,Start,SEO,3972992,2.516995e-07
7,100004330nan0,100004330,0,nan,1,SEO,End,1976015,5.060690e-07
8,100004606114473205.01,100004606,1,114473205.0,0,Start,Direct+Other,3972992,2.516995e-07
9,100004606114473205.01,100004606,1,114473205.0,1,Direct+Other,SEO,1281018,7.806292e-07


In [21]:
# Check totals, that porbability is 1
df_test = Transitions.groupby('Channel').agg({'probability': 'sum'})
df_test

,probability
Channel,
Digital Event,1.0
Direct+Other,1.0
Display,1.0
Email,1.0
Friendly URL,1.0
Paid Search,1.0
SEO,1.0
Social,1.0
Social-Paid,1.0


In [22]:
df_transition_prob = Transitions.groupby(['Channel','Transition']).agg({'probability':'sum'}).reset_index()
df_transition_prob

,Channel,Transition,probability
0,Digital Event,Conv,0.133333
1,Digital Event,Direct+Other,0.177778
2,Digital Event,Email,0.044444
3,Digital Event,End,0.288889
4,Digital Event,Friendly URL,0.088889
5,Digital Event,Paid Search,0.200000
6,Digital Event,SEO,0.044444
7,Digital Event,Social,0.022222
8,Direct+Other,Conv,0.034686
9,Direct+Other,Digital Event,0.000009


In [27]:
#Fit DataFrame to code
df_transition_prob.rename(columns={'Channel':'t','Transition':'t_plus_1'}, inplace=True)

df_transition_prob

,t,t_plus_1,probability
0,Digital Event,Conv,0.133333
1,Digital Event,Direct+Other,0.177778
2,Digital Event,Email,0.044444
3,Digital Event,End,0.288889
4,Digital Event,Friendly URL,0.088889
5,Digital Event,Paid Search,0.200000
6,Digital Event,SEO,0.044444
7,Digital Event,Social,0.022222
8,Direct+Other,Conv,0.034686
9,Direct+Other,Digital Event,0.000009


### Calculate Total Conversion Probability

In [24]:
def print_node(node, debug=False):
    # Print each node as it's processed. For debugging purposes.
    
    if not debug:
        return
    
    if node['t_plus_1'] in ['End', 'Conv']:
        node_type = 'Leaf'
    else:
        node_type = 'Parent'
    
    print('%s > %s' % (node['t'], node['t_plus_1']))
    print('Type: %s' % node_type)
    print('Prob: %0.2f' % node['probability'])
    print('----------------------------')

### ASSUMES THE TRANSITION DATAFRAME REPRESENTS A "DAG"
###### https://www.python-course.eu/graphs_python.php  for solution ideas...

In [25]:
calculated_node_probabilities = dict()

def calc_conversion_probability(starting_state, df_transitions, cum_probability, calculated_nodes, debug=True):
    # Calculates the cumulative probability of reaching a conversion, given a starting state.
    # Assumes the transition dataframe represents a Directed Acyclic Graph (DAG)
        
    # Get the transition probabilities for the starting state we're evaluating
    df_nodes = df_transitions[df_transitions['t'] == starting_state]

    
    # Loop through the starting states and either return the probability for 
    # a leaf node, or recurse to keep following the tree.
    
    node_conversion_probability = 0
    
    child_node_proabilities = []
    
    for index, row in df_nodes.iterrows():
        
        # These are leaf nodes: either an exit or conversion
        if row['t_plus_1'] == 'End':
            print_node(row, debug)
            child_node_proabilities.append(0)
        
        elif row['t_plus_1'] == 'Conv':
            print_node(row, debug)
            child_node_proabilities.append(row['probability'])
        
        # This is a parent node: Keep following the rabbit hole
        else:
            
            # Have we cached the total probability for this node???
            if row['t_plus_1'] in calculated_nodes:
                if debug:
                    print('Cache Hit for %s! Cum probability from child: %0.2f' % (row['t_plus_1'], calculated_nodes[row['t_plus_1']]))
                child_probability = calculated_nodes[row['t_plus_1']]
            
            # No cached value found. We'll walk through the tree to calculated the value.
            else:
                # Recursive call
                child_probability = calc_conversion_probability(row['t_plus_1'], 
                                                                df_transitions, 
                                                                cum_probability + row['probability'],
                                                                calculated_nodes, 
                                                                debug)
                node_conversion_prob = child_probability * row['probability']
                
            print_node(row, debug)
            child_node_proabilities.append(node_conversion_prob)
                    
            if debug:
                print('%s > %s' % (row['t'], row['t_plus_1']))
                print('Cum Prob from Child : %0.2f' % child_probability)
                print('Prob to Child Node  : %0.2f' % row['probability'])
                print('Node Conv Proability: %0.2f' % node_conversion_prob)
                print('----------------------------')
    
    total_node_probability = sum(child_node_proabilities)
    if debug:
        print('Node Conversion Probability for %s: %0.2f' % (starting_state, total_node_probability))
        print('----------------------------')
    
    # We'll cache the calculated total probability for the node, so we don't have to calculate it again.
    calculated_node_probabilities[starting_state] = total_node_probability
        
    return total_node_probability

In [28]:
starting_node = 'SEO'
print('====== START DEBUG PRINT ======')
total_probability = calc_conversion_probability(starting_node, df_transition_prob, 0, calculated_node_probabilities)
print('====== END DEBUG PRINT ======')
print(' ')
print('Total Conversion Probability from %s: %0.2f' % (starting_node, total_probability))

====== START DEBUG PRINT ======
SEO > Conv
Type: Leaf
Prob: 0.01
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
-----

Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------

Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------

Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------

Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------

Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------
Direct+Other > Conv
Type: Leaf
Prob: 0.03
----------------------------
Digital Event > Conv
Type: Leaf
Prob: 0.13
----------------------------

RuntimeError: maximum recursion depth exceeded while calling a Python object

##### No Final output, error received is "RuntimeError: maximum recursion depth exceeded while calling a Python object"